# 初用pytorch

## 环境引入

In [1]:
import torch
import torch.nn as nn
import numpy as np


## 基本outograd（自动求导）1

In [2]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)


## 基本outograd（自动求导）2

In [3]:
x = torch.randn(10, 3)
y = torch.randn(10, 2)
# build a fully connected layer
linear = nn.Linear(3, 2)
print('w:', linear.weight )
print('b:', linear.bias )

w: Parameter containing:
tensor([[ 0.5266,  0.0328, -0.1934],
        [-0.3691, -0.2857,  0.2334]], requires_grad=True)
b: Parameter containing:
tensor([0.5375, 0.2988], requires_grad=True)


In [4]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)
pred = linear(x)

# compute loss
loss = criterion(pred, y)
print('loss:', loss.item())
loss.backward()
print('DL/Dw:', linear.weight.grad)
print('DL/Db:', linear.bias.grad)

optimizer.step()

pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 round optimization:', loss.item())



loss: 2.677647113800049
DL/Dw: tensor([[ 1.3533, -0.1146, -0.0117],
        [-0.6534, -0.0701,  0.0197]])
DL/Db: tensor([0.3794, 0.0167])
loss after 1 round optimization: 2.65360951423645


## 引入一个pipeline（深度学习流程）
（原标题，但内容好像只引入了一个torchvision包的datasets）

In [4]:
import torchvision
import torchvision.transforms as transforms
train_dataset = torchvision.datasets.CIFAR10(root='../../datasets/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

Files already downloaded and verified


In [6]:
image, label = train_dataset[0]
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=64,
                                          shuffle=True)
data_iter = iter(train_loader)
images, labels = data_iter.next()
print(np.shape(images), len(labels))
# for images, labels in train_loader:
#     pass

torch.Size([64, 3, 32, 32]) 64


## 引入预训练的模型
如果想要fintune的话，将模型所有参数的requires_grad改为False，这样反向传播时就不会进行求到，只对最后一层全连接层进行处理

In [10]:
resnet = torchvision.models.resnet18(pretrained=True)


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/liang/.cache/torch/checkpoints/resnet18-5c106cde.pth
100.0%


In [13]:
for param in resnet.parameters():
    param.requires_grad = False

In [14]:
resnet.fc = nn.Linear(resnet.fc.in_features, 100)

In [16]:
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())

torch.Size([64, 100])


In [18]:

torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

In [19]:
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>